In [2]:
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
genres=['concert', 'festival', 'spectacle-et-comedie-musicale', 'humour-et-one-wo-man-show', 'sport', 'parc', 'exposition-et-musee', 'theatre', 'cirque', 'salon', 'danse', 'classique-et-opera', 'loisirs-et-tourisme', 'soiree']

In [ ]:
def new_str(string):
    return string.replace('[','').replace(']','').replace("'",'').replace(',','-')

In [12]:
genre='festival'
url="https://www.ticketmaster.fr/fr/"+genre+"/p/1"
request_text = request.urlopen(url).read()
htmlpage= BeautifulSoup(request_text, "html")

In [13]:
print(htmlpage.find("title").text)

Billetterie Concert | Billet & Réservation Place Concerts | Ticketmaster


In [3]:
genres=['concert', 'festival', 'spectacle-et-comedie-musicale', 'humour-et-one-wo-man-show', 'sport', 'parc', 'exposition-et-musee', 'theatre', 'cirque', 'salon', 'danse', 'classique-et-opera', 'loisirs-et-tourisme', 'soiree']
events=[]
for genre in genres:
    url="https://www.ticketmaster.fr/fr/"+genre+"/p/1"
    request_text = request.urlopen(url).read()
    htmlpage= BeautifulSoup(request_text, "html")
    if (htmlpage.find('a',{'name':'lienDernierePage'}) == None):
        lastpage=1
    else:
        lastpage = int(htmlpage.find('a',{'name':'lienDernierePage'}).text)
    for i in range(1,lastpage+1):
        url="https://www.ticketmaster.fr/fr/"+genre+"/p/"+str(i)
        request_text = request.urlopen(url).read()
        htmlpage= BeautifulSoup(request_text, "html")
        for item in htmlpage.findAll('article',{'class':'bloc-result bloc-billet-fr'}):
            event = {}
            priceinfo=item.find('div',{'class':'bloc-result-ticket-info'}).find('strong')
            if (priceinfo != None):
                place = item.find('p',{'class':'bloc-result-location'}).text.lstrip('\n').split(' - ')
                event['Date']=item.find('p',{'class':'bloc-result-when'}).text.replace('é','e')
                event['Lieu']=place[0]
                event['Ville']=place[len(place)-1]
                event['Type']=item.find('p',{'class':'bloc-result-type'}).text.replace('\n','')
                event['Prix']=priceinfo.text.lstrip('\n').replace(' €','')
                event['Artiste']=[]
                for artistes in item.findAll('p',{'class':'bloc-result-artist'}):               
                    for artiste in artistes.text.split('\n'):
                        if artiste != ',' and artiste != 'Artiste :' and artiste != '':
                            event['Artiste'].append(artiste.lstrip('Artiste : '))
                event['Image']=item.find('img')['src']
                event['Genre']=genre
                events.append(event)
df = pd.DataFrame(events)
df

,Date,Lieu,Ville,Type,Prix,Artiste,Image,Genre
0,28 avr. 2023,ACCOR ARENA,PARIS,CONCERT - VARIETE INTERNATIONALE,39.00,[IBIZA EXPERIENCE],https://static.ticketmaster.fr/static/images/v...,concert
1,1 au 3 juin 2023,ACCOR ARENA,PARIS,CONCERT - VARIETE ET CHANSON FRANCAISE,35.00,[-M-],https://static.ticketmaster.fr/static/images/v...,concert
2,2 dec. 2023,PARIS LA DEFENSE ARENA,NANTERRE,CONCERT - POP ROCK FOLK,39.00,[STROMAE],https://static.ticketmaster.fr/static/images/v...,concert
3,18 au 19 juin 2023,ZENITH PARIS,PARIS,CONCERT - POP ROCK FOLK,60.50,"[THE BLACK KEYS, SPOON]",https://static.ticketmaster.fr/static/images/v...,concert
4,24 au 25 juin 2023,STADE PIERRE MAUROY,VILLENEUVE D ASCQ,CONCERT - RAP HIP-HOP SLAM,40.00,[SOPRANO],https://static.ticketmaster.fr/static/images/v...,concert
...,...,...,...,...,...,...,...,...
10618,10 juin au 23 sept. 2023,JARDINS DU CHATEAU DE VERSAILLES,VERSAILLES,LOISIRS ET TOURISME - VISITE ET EXCURSION,29.70,[LES GRANDES EAUX NOCTURNES],https://static.ticketmaster.fr/static/images/v...,loisirs-et-tourisme
10619,31 dec. 2022,PALAIS DE L'EPI D'OR,VILLEJUIF,SOIREE - REVEILLON,25.00,[REVEILLON 2023],https://static.ticketmaster.fr/static/images/v...,soiree
10620,31 dec. 2022,New Morning,PARIS,SOIREE - REVEILLON,55.00,[LE GRAND REVEILLON DU NOUVEL AN],https://static.ticketmaster.fr/static/images/v...,soiree
10621,6 fevr. 2023,LES FOLIES BERGERE,PARIS,SOIREE - SOIREE MUSIQUE,20.00,"[CORPS A COEURS, LOUANE, BERENGERE KRIEF, JOYC...",https://static.ticketmaster.fr/static/images/v...,soiree


In [ ]:
#Puisque lorsqu'on récupère les artistes, on les récupère dans un tableau, 
#On va les transformer en string en retirant les crochets, les apostrophes et on remplace les virgules par un tiret 
for i in range(len(df)):
    df['Artiste'][i]= new_str(df['Artiste'][i])

In [8]:
#On crée un fichier csv pour chaque genre
for genre in genres:
    df_temp=df.loc[df['Genre'] == genre]
    del df_temp['Genre'] #On supprime la colonne genre puisqu'elle servait juste à grouper les données
    df_temp.to_csv('events/'+genre+'.csv',index=False)